# Sequence to Sequence Models

This chapter introduces you to two applications of RNN models: Text Generation and Neural Machine Translation. You will learn how to prepare the text data to the format needed by the models. The Text Generation model is used for replicating a character's way of speech and will have some fun mimicking Sheldon from The Big Bang Theory. Neural Machine Translation is used for example by Google Translate in a much more complex model. In this chapter, you will create a model that translates Portuguese small phrases into English.

# (1) Sequence to Sequence Models

## Sequence to sequence
Possible architectures:
- Many inputs with one output
    - Sentiment analysis
    - Classification
- Many inputs to many outputs
    - Text generation
    - Neural Machine Translation (NMT)

## Text generation: example
Text generation: example

In [ ]:
# Pre-trained model
model.generate_sheldon_phrase()

## Text generation: modeling
How to build text generation models:
- Decide if a token will be characters or words
    - Words demands very large datasets (hundred of millions sentences)
    - Chars can be trained faster, but can generate typos
- Prepare the data
    - Build training sample with (past tokens, next token) examples
- Design the model architecture
    - Embedding layer, number of layers, etc.
- Train and experiment

## NMT: example
Neural Machine Translation: example

In [ ]:
# Pre-trained model
model.translate("Vamos jogar futebol?")

## NMT: modeling
How to build `NMT` models:
- Get a sample of translated sentences
    - For example, the **Anki project**
- Prepare the data
    - Tokenize input language sentences
    - Tokenize output language sentences
- Design the model architecture
    - Encoder and decoder
- Train and experiment

## Chapter outliner
In this chapter:
- Text Generation
    - Use pre-trained model to generate a sentence
    - Learn to prepare the data and build the model
- Neural Machine Translation (NMT)
    - All-in-one NMT model

# Exercise I: Text generation examples

In this exercise, you are going to experiment on two pre-trained models for text generation.

The first model will generate one phrase based on the character Sheldon of The Big Bang Theory TV show, and the second model will generate a Shakespeare poems up to 400 characters.

The models are loaded on the `sheldon_model` and `poem_model` variables. Also, two custom functions to help generate text are available: `generate_sheldon_phrase()` and `generate_poem()`. Both receive the pre-trained model and a context string as parameters.

### Instructions

- Use pre-defined function `generate_sheldon_phrase()` with parameters `sheldon_model` and `sheldon_context` and store the output in the `sheldon_phrase` variable.
- Print the obtained phrase.
- Store the given text into the `poem_context` variable.
- Print the poem generated by applying the function `generate_poem()` with the `poem_model` and `poem_context` parameters.


In [ ]:
# Context for Sheldon phrase
sheldon_context = "I’m not insane, my mother had me tested. "

# Generate one Sheldon phrase
sheldon_phrase = generate_sheldon_phrase(sheldon_model, sheldon_context)

# Print the phrase
print(sheldon_phrase)

# Context for poem
poem_context = "May thy beauty forever remain"

# Print the poem
print(generate_poem(poem_model, poem_context))

# Exercise II: NMT example

This exercise aims to build on the sneak peek you got of NMT at the beginning of the course. You will continue to translate Portuguese small phrases into English.

Some sample sentences are available on the `sentences` variable and are printed on the console.

Also, a pre-trained model is available on the `model` variable and you will use two custom functions to simplify some steps:

- `encode_sequences()`: Change texts into sequence of numerical indexes and pad them.
- `translate_many()`: Uses the pre-trained model to translate a list of sentences from Portuguese into English. Later you will code this function yourself.

For more details on the functions, use `help()`. The package `pandas` is loaded as `pd`.

### Instructions

- Use the `encode_sequences()` function to pre-process the texts and save the results in the `X` variable.
- Translate the `sentences` using the `translate_many()` function by passing `X` as a parameter.
- Create a `pd.DataFrame()` with the original and translated lists as columns.
- Print the data frame.


In [ ]:
# Transform text into sequence of indexes and pad
X = encode_sequences(sentences)

# Print the sequences of indexes
print(X)

# Translate the sentences
translated = translate_many(model, X)

# Create pandas DataFrame with original and translated
df = pd.DataFrame({'Original': sentences, 'Translated': translated})

# Print the DataFrame
print(df)